In [202]:
from datetime import datetime
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from markdownify import markdownify 
import folium
import io




# endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """
#defaultView:Map

SELECT DISTINCT ?item ?itemLabel ?location ?locationLabel ?latitude ?longitude ?geo WHERE {
  VALUES ?type { wd:Q1007870 wd:Q33506 wd:Q166118 wd:Q2668072 wd:Q667276 
                 wd:Q464980 wd:Q56190453 }
  #art gallery, museum, archive, collection, art exhibition, 
  # exhibition, memorial
  
  VALUES ?places { wd:Q1715 } 

  # Find locations that match ?places directly OR one level above/below
  ?place (wdt:P131|^wdt:P131) ?places. 

  # Find items that are in those locations
  ?item wdt:P31 ?type;  # Must be an instance of a relevant cultural type
        wdt:P276|wdt:P131 ?place.  # Located in one of the places
  
  ?item wdt:P625 ?geo.  # Get coordinates

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,[AUTO_LANGUAGE]". }

  BIND(geof:latitude(?geo) AS ?latitude).
  BIND(geof:longitude(?geo) AS ?longitude).
}
"""



def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

locations = []

def get_items():

#    print("<link rel='stylesheet' href='./masonrydesign.css'>")
    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else ''
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['placeLabel']['value'] + ', ' + item['countryLabel']['value'] if 'placeLabel' in item else ''
      item_url=item['item']['value'] if 'item' in item else ''
      image_url=item['image']['value'] if 'image' in item else ''

      locations.append((title, latitude, longitude, image_url, item_url, place, alt))






In [ ]:
get_items()

m_hannover = folium.Map(location=[52.38, 9.74], zoom_start=12)

for name, lat, lon, url, item, place, alt in locations:
    folium.Marker(
        location=[lat, lon],
        popup=f"""<h3><a href='{item}' target='_blank'>{name}</a></h3>
        {place}""",
        #<img src='{url}' alt='{alt}' style='width:200px;height:200px;object-fit:contain;'>
        #<a href='{url}' target='_blank'>Full size image</a>""",  
        tooltip=name  # Hover label
    ).add_to(m_hannover)

print(""" 
::: {.content-visible when-format='html'}

## Hannover 
      
[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flocation%20%3FlocationLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20WHERE%20%7B%0A%20%20VALUES%20%3Ftype%20%7B%20wd%3AQ1007870%20wd%3AQ33506%20wd%3AQ166118%20wd%3AQ2668072%20wd%3AQ667276%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ464980%20wd%3AQ56190453%20%7D%0A%20%20%23art%20gallery%2C%20museum%2C%20archive%2C%20collection%2C%20art%20exhibition%2C%20%0A%20%20%23%20exhibition%2C%20memorial%0A%20%20%0A%20%20VALUES%20%3Fplaces%20%7B%20wd%3AQ1715%20%7D%20%0A%0A%20%20%23%20Find%20locations%20that%20match%20%3Fplaces%20directly%20OR%20one%20level%20above%2Fbelow%0A%20%20%3Fplace%20(wdt%3AP131%7C%5Ewdt%3AP131)%20%3Fplaces.%20%0A%0A%20%20%23%20Find%20items%20that%20are%20in%20those%20locations%0A%20%20%3Fitem%20wdt%3AP31%20%3Ftype%3B%20%20%23%20Must%20be%20an%20instance%20of%20a%20relevant%20cultural%20type%0A%20%20%20%20%20%20%20%20wdt%3AP276%7Cwdt%3AP131%20%3Fplace.%20%20%23%20Located%20in%20one%20of%20the%20places%0A%20%20%0A%20%20%3Fitem%20wdt%3AP625%20%3Fgeo.%20%20%23%20Get%20coordinates%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND(geof%3Alatitude(%3Fgeo)%20AS%20%3Flatitude).%0A%20%20BIND(geof%3Alongitude(%3Fgeo)%20AS%20%3Flongitude).%0A%7D)
""")   



 
::: {.content-visible when-format='html'}

## Hannover 
      
[Wikidata Link](https://query.wikidata.org/#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flocation%20%3FlocationLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20WHERE%20%7B%0A%20%20VALUES%20%3Ftype%20%7B%20wd%3AQ1007870%20wd%3AQ33506%20wd%3AQ166118%20wd%3AQ2668072%20wd%3AQ667276%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ464980%20wd%3AQ56190453%20%7D%0A%20%20%23art%20gallery%2C%20museum%2C%20archive%2C%20collection%2C%20art%20exhibition%2C%20%0A%20%20%23%20exhibition%2C%20memorial%0A%20%20%0A%20%20VALUES%20%3Fplaces%20%7B%20wd%3AQ1715%20%7D%20%0A%0A%20%20%23%20Find%20locations%20that%20match%20%3Fplaces%20directly%20OR%20one%20level%20above%2Fbelow%0A%20%20%3Fplace%20%28wdt%3AP131%7C%5Ewdt%3AP131%29%20%3Fplaces.%20%0A%0A%20%20%23%20Find%20items%20that%20are%20in%20those%20locations%0A%20%20%3Fitem%20wdt%3AP31%20%3Ftype%3B%20%20%23%20Must%20be%20an%20instance%20of%20a%20relevant%

In [204]:

m_hannover


In [205]:
# m_berlin = folium.Map(location=[52, 10], zoom_start=2)

# locations = [
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', '', ''),
#     ('', '', ''),
#     ]

# for name, lat, lon, url in locations:
#     folium.Marker(
#         location=[lat, lon],
#         popup=f"""<h3><a href='{item}' target='_blank'>{name}</a></h3>
#         {place}""",
#         #<img src='{url}' alt='{alt}' style='width:200px;height:200px;object-fit:contain;'>
#         #<a href='{url}' target='_blank'>Full size image</a>""",  
#         tooltip=name  # Hover label
#     ).add_to(m_hannover)

# print(""" 
# ::: {.content-visible when-format='html'}

# ## Hannover 
      
# [Wikidata Link](https://query.wikidata.org/#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flocation%20%3FlocationLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20WHERE%20%7B%0A%20%20VALUES%20%3Ftype%20%7B%20wd%3AQ1007870%20wd%3AQ33506%20wd%3AQ166118%20wd%3AQ2668072%20wd%3AQ667276%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ464980%20wd%3AQ56190453%20%7D%0A%20%20%23art%20gallery%2C%20museum%2C%20archive%2C%20collection%2C%20art%20exhibition%2C%20%0A%20%20%23%20exhibition%2C%20memorial%0A%20%20%0A%20%20VALUES%20%3Fplaces%20%7B%20wd%3AQ1715%20%7D%20%0A%0A%20%20%23%20Find%20locations%20that%20match%20%3Fplaces%20directly%20OR%20one%20level%20above%2Fbelow%0A%20%20%3Fplace%20%28wdt%3AP131%7C%5Ewdt%3AP131%29%20%3Fplaces.%20%0A%0A%20%20%23%20Find%20items%20that%20are%20in%20those%20locations%0A%20%20%3Fitem%20wdt%3AP31%20%3Ftype%3B%20%20%23%20Must%20be%20an%20instance%20of%20a%20relevant%20cultural%20type%0A%20%20%20%20%20%20%20%20wdt%3AP276%7Cwdt%3AP131%20%3Fplace.%20%20%23%20Located%20in%20one%20of%20the%20places%0A%20%20%0A%20%20%3Fitem%20wdt%3AP625%20%3Fgeo.%20%20%23%20Get%20coordinates%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND%28geof%3Alatitude%28%3Fgeo%29%20AS%20%3Flatitude%29.%0A%20%20BIND%28geof%3Alongitude%28%3Fgeo%29%20AS%20%3Flongitude%29.%0A%7D)
# """)   

In [206]:
print("""
:::
""")



:::



In [207]:

# Generate image from HTML
img_data = m_hannover._to_png(5)
img = Image.open(io.BytesIO(img_data))

img.save('images/museumsmap_hannover.png')

print(""" 
:::{.content-visible unless-format='html'}
      
![](images/museumsmap_hannover.png)

:::
""")


 
:::{.content-visible unless-format='html'}
      
![](images/museumsmap_hannover.png)

:::



In [208]:
#get_graph()